In [82]:
# coding: utf-8
import sys
import os
from tqdm import tqdm
sys.path.append(os.pardir)  # 为了导入父目录的文件而进行的设定
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image, ImageFilter
import seaborn as sns

si=7
bd=6

In [83]:
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

def resize(img):
    img = img.reshape(28, 28)  # 把图像的形状变为原来的尺寸
    t = np.zeros((28,28))
    for i in range(28):
        for j in range(28):
            t[i][j] = img[i][j]

    pix = np.zeros((si, si))
    blo=28//si
    for i in range(si):
        for j in range(si):
            for a in range(blo):
                for b in range(blo):
                    if t[i*blo+a][j*blo+b]>0:
                        pix[i][j] += 1
#     img_show(img)
    for i in range(si):
        for j in range(si):
            pix[i][j]=1 if pix[i][j]>bd else 0
    pix=pix.reshape(si*si).astype(np.uint32)
    return pix

In [84]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

pclass=np.zeros(10)
numclass=np.zeros(10)
for i in range(len(x_train)):
    numclass[t_train[i]]+=1
        
for i in range(10):
    pclass[i]=numclass[i]/len(x_train)
print(pclass)

[0.09871667 0.11236667 0.0993     0.10218333 0.09736667 0.09035
 0.09863333 0.10441667 0.09751667 0.09915   ]


In [85]:
feature=[]
p=np.ones((si*si,10))
for imgIndex in tqdm(range(len(x_train))):
    img = x_train[imgIndex]  # 改变序号可以读取不同的图片
    label = t_train[imgIndex]  # 改变序号可以读取不同的图片
    pix=resize(img)
    for i in range(si*si):
        p[i][label]+=pix[i]
    feature.append(pix)

100%|██████████| 60000/60000 [01:07<00:00, 887.36it/s]


In [86]:
for i in range(10):
    for j in range(si*si):
        p[j][i]/=numclass[i]+2
p[15]

array([0.12725738, 0.00133452, 0.08775168, 0.09897277, 0.12491444,
       0.07154711, 0.04831081, 0.35934259, 0.09550658, 0.10250378])

In [88]:
ans=0
for index in tqdm(range(len(x_test))):
    bayes=np.ones(10)
    pix=resize(x_test[index])
#     print(pix.reshape(si,si))
    for i in range(10):
        t=1.0
        for j in range(si*si):
            t*=p[j][i] if pix[j]==1 else 1-p[j][i]
#         print(t)
        bayes[i]*=t*pclass[i]
        t=.0
        for j in range(si*si):
            t+=p[j][i]
        bayes[i]/=t
#     print(bayes)
    if t_test[index]==np.argmax(bayes):
        ans+=1
#     print(t_test[index],np.argmax(bayes))


100%|██████████| 10000/10000 [00:35<00:00, 285.13it/s]

In [89]:
ans/len(t_test)

0.7411

In [90]:
ans[:100]

TypeError: 'int' object is not subscriptable